In [ ]:
!pip install meteostat -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily

In [9]:
cities = {'Tokyo':[35.652832,139.839478]}
cities.values()

list(cities.values())[0]

[34.543896, 69.160652]

In [10]:
list(cities.keys())[0]

'Kabul'

In [14]:
# settaggio periodo di forescast (1settimana)

start = datetime(2022, 1, 1)
end = datetime(2025, 2, 3)

cities = {'Tokyo':[35.652832,139.839478]}

# creando un punto per la città di Tokyo
city = Point(list(cities.values())[0][0],list(cities.values())[0][1], 20)

# dati meteo di Tokyo

data = Daily(city, start, end)
data = data.fetch()
data['city'] = list(cities.keys())[0]

data.head()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,city
time,,,,,,,,,,,
2022-01-01,2.6,-1.4,6.9,0.0,NaN,314.0,17.1,NaN,1018.9,NaN,Tokyo
2022-01-02,3.2,-0.7,7.9,0.0,NaN,330.0,8.8,NaN,1020.5,NaN,Tokyo
2022-01-03,5.6,1.4,9.9,0.0,NaN,347.0,9.8,NaN,1016.6,NaN,Tokyo
2022-01-04,5.7,1.5,10.9,0.0,NaN,331.0,11.7,NaN,1012.9,NaN,Tokyo
2022-01-05,3.7,0.7,7.9,0.0,NaN,337.0,16.6,NaN,1020.9,NaN,Tokyo


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1130 entries, 2022-01-01 to 2025-02-03
Freq: D
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tavg    1130 non-null   float64
 1   tmin    1130 non-null   float64
 2   tmax    1130 non-null   float64
 3   prcp    1117 non-null   float64
 4   snow    0 non-null      float64
 5   wdir    1130 non-null   float64
 6   wspd    1130 non-null   float64
 7   wpgt    0 non-null      float64
 8   pres    1130 non-null   float64
 9   tsun    0 non-null      float64
 10  city    1130 non-null   object 
dtypes: float64(10), object(1)
memory usage: 105.9+ KB


In [23]:
import streamlit as st
import pandas as pd
import numpy as np
from prophet import Prophet
import plotly.express as px

In [24]:
!pip install prophet -q

In [25]:
def get_weather_data():
   city = Point(35.652832, 139.839478, 20)
   start = datetime(2022, 1, 1)
   end = datetime(2025, 2, 3)
   
   data = Daily(city, start, end).fetch()
   data = data.reset_index()
   return data

def prepare_prophet_data(data, target_col):
   prophet_data = data[['time', target_col]].copy()
   prophet_data.columns = ['ds', 'y']
   prophet_data = prophet_data.dropna()
   return prophet_data

def make_forecast(data, target_col, days=7):
   prophet_data = prepare_prophet_data(data, target_col)
   
   model = Prophet(daily_seasonality=True)
   model.fit(prophet_data)
   
   future = model.make_future_dataframe(periods=days)
   forecast = model.predict(future)
   
   return forecast

def main():
   st.title("Tokyo Weather Forecast")
   
   data = get_weather_data()
   
   st.sidebar.header("Forecast Settings")
   forecast_days = st.sidebar.slider("Forecast Days", 1, 14, 7)
   target_column = st.sidebar.selectbox(
       "Select Weather Parameter",
       ['tavg', 'tmin', 'tmax', 'prcp', 'wspd']
   )
   
   forecast = make_forecast(data, target_column, forecast_days)
   
   st.subheader("Historical Data")
   fig_historical = px.line(data, x='time', y=target_column,
                          title=f'Historical {target_column} in Tokyo')
   st.plotly_chart(fig_historical)
   
   st.subheader("Forecast")
   fig_forecast = px.line(forecast, x='ds', y=['yhat', 'yhat_lower', 'yhat_upper'],
                         title=f'{target_column} Forecast for Next {forecast_days} Days')
   st.plotly_chart(fig_forecast)
   
   st.subheader("Forecast Values")
   forecast_tail = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(forecast_days)
   forecast_tail.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']
   st.dataframe(forecast_tail)

if __name__ == "__main__":
   main()

2025-02-03 14:58:35.842 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-03 14:58:35.843 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-03 14:58:35.896 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-03 14:58:35.897 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-03 14:58:35.898 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-03 14:58:35.898 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-03 14:58:35.898 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-03 14:58:35.899 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar